In [1]:
# modules we'll use
import pandas as pd
import numpy as np
import chardet
import datetime
import fuzzywuzzy
from fuzzywuzzy import process

# TODO: read in all your data here by give file path
path_to_data = "./data/Building_Permits.csv"

data_encoding = "utf-8"

# Try identify file encoding and read data in
# with open(path_to_data, 'rb') as rawdata:
#    data_encoding = chardet.detect(rawdata.read(100000))["encoding"]
input_data = pd.read_csv(path_to_data, encoding=data_encoding)

# Ensure all columns is shown when printed
pd.options.display.max_columns = None
pd.options.display.width=None

# set seed for reproducibility
np.random.seed(0) 

In [2]:
# Analyse Data - run if necessary
print("\nDescribe " + "=" * 150)
print(input_data.describe())

print("\nHead " + "=" * 150)
print(input_data.head())

print("\nNumber of NaNs " + "=" * 150)
print(input_data.isnull().sum())


Describe ======================================================================================================================================================
         Permit Type  Street Number          Unit  Number of Existing Stories  \
count  198900.000000  198900.000000  29479.000000               156116.000000   
mean        7.522323    1121.728944     78.517182                    5.705773   
std         1.457451    1135.768948    326.981324                    8.613455   
min         1.000000       0.000000      0.000000                    0.000000   
25%         8.000000     235.000000      0.000000                    2.000000   
50%         8.000000     710.000000      0.000000                    3.000000   
75%         8.000000    1700.000000      1.000000                    4.000000   
max         8.000000    8400.000000   6004.000000                   78.000000   

       Number of Proposed Stories  Estimated Cost  Revised Cost  \
count               156032.000000    1.608

In [3]:
# Process NaN Values
nan_processed_data = input_data

# nan_processed_data = input_data.dropna() # Drop rows with NaN
# nan_processed_data = input_data.dropna(axis=1) # Drop columns with NaN
# nan_processed_data = input_data.fillna(0) # Fill NaN with some value
# nan_processed_data = input_data.fillna(method='bfill') # Backfill NaN with next valid value

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html

In [4]:
# Data Cleaning
cleaned_data = nan_processed_data

# nan_processed_data['Country'] = nan_processed_data['Country'].str.lower() # convert to lower case
# nan_processed_data['Country'] = nan_processed_data['Country'].str.strip() # remove trailing white spaces

In [5]:
# Data Cleaning - Fuzzy Values helper

def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
    strings = df[column].unique()  
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > min_ratio
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    rows_with_matches = df[column].isin(close_matches) 
    df.loc[rows_with_matches, column] = string_to_match  # replace close matches

In [7]:
# Data Cleaning - Fuzzy Values

series_to_check = cleaned_data["column_name"].unique()
fuzzy_matches = fuzzywuzzy.process.extract("string value to compare to", series_to_check, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
print(fuzzy_matches)

# Finally replace if needed
# replace_matches_in_column(df=cleaned_data, column='column_name', string_to_match="string_value", min_ratio=70)

[('1 family dwelling', 94), ('2 family dwelling', 94), ('building materials', 48), ('filling/service stn', 41), ('clinics-medic/dental', 40), ('fence/retaining wall', 40), ('social care facility', 40), ('r-3(dwg) nursing', 40), ('library', 36), ('moving & storage', 34)]
